In [412]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.graph_objs import *
import numpy as np
init_notebook_mode(connected=True)
import numpy as np
import geopandas as gp
import plotly.plotly as py_cloud
# https://github.com/plotly/dash-uber-rides-demo/blob/master/app.py

In [371]:
# read in population data
df = pd.read_csv('airbnbwards.csv')

# Read in geojson file of DC ward polygons
with open("dc-ward-map-overlay.geojson") as f:
    wards = geojson.load(f)

In [372]:
# Format column names
for col in df.columns:
    df[col] = df[col].astype(str)
# Drop columns that won't be used
df.drop(['room_id', 'host_id', 'room_type', 'neighborhood', 'reviews', 'accommodates', 'bedrooms', 'minstay'],
             inplace = True, axis=1)
print(df)

  Unnamed: 0    Ward overall_satisfaction   price     latitude     longitude
0          1  Ward 1           1.98806366  228.33  38.92284323  -77.03190794
1          2  Ward 2          1.882616487  292.31   38.9071684  -77.03522891
2          3  Ward 3          1.543050431  262.08  38.93525823  -77.07349277
3          4  Ward 4   1.7681159419999999  182.25  38.95797802  -77.02670979
4          5  Ward 5          2.005803571  189.58  38.93042564    -77.001141
5          6  Ward 6          1.873062016   340.9  38.89092596  -76.99809961
6          7  Ward 7          1.567982456  198.26  38.89386739  -76.96842268
7          8  Ward 8          1.705513784  138.03  38.86533188  -76.96732552


In [373]:
# initialize a dictionary
geo_dict = {}

for x in range(len(wards['features'])):
    # I ignore the last eleven characters in the name since the geojson file includes ' County, CO' in the county names and the population data does not
    name = wards['features'][x]['properties']['name']
    if name in df['Ward'].unique():
        geo_dict[name] = wards['features'][x]
    else:
        print ('not in: ', name)

In [374]:
# Once I have my dictionary of geolocation data in order, I want to make it into a pandas series to make it easy to 
# join to my airbnbwards dataframe
ser = pd.Series(list(geo_dict.values()), index = geo_dict.keys())
ser.name = 'coordinates'
print(ser)

Ward 1    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 2    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 3    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 4    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 5    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 6    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 7    {'type': 'Feature', 'geometry': {'type': 'Poly...
Ward 8    {'type': 'Feature', 'geometry': {'type': 'Poly...
Name: coordinates, dtype: object


In [375]:
# Join geolocation data to airbnbwards dataframe
df = df.join(ser, on='Ward')
print(df)

  Unnamed: 0    Ward overall_satisfaction   price     latitude     longitude  \
0          1  Ward 1           1.98806366  228.33  38.92284323  -77.03190794   
1          2  Ward 2          1.882616487  292.31   38.9071684  -77.03522891   
2          3  Ward 3          1.543050431  262.08  38.93525823  -77.07349277   
3          4  Ward 4   1.7681159419999999  182.25  38.95797802  -77.02670979   
4          5  Ward 5          2.005803571  189.58  38.93042564    -77.001141   
5          6  Ward 6          1.873062016   340.9  38.89092596  -76.99809961   
6          7  Ward 7          1.567982456  198.26  38.89386739  -76.96842268   
7          8  Ward 8          1.705513784  138.03  38.86533188  -76.96732552   

                                         coordinates  
0  {'type': 'Feature', 'geometry': {'type': 'Poly...  
1  {'type': 'Feature', 'geometry': {'type': 'Poly...  
2  {'type': 'Feature', 'geometry': {'type': 'Poly...  
3  {'type': 'Feature', 'geometry': {'type': 'Poly...  
4  {

In [315]:
mapbox_access_token = 'pk.eyJ1IjoiY21jNDU0IiwiYSI6ImNqZ2kyNzZoNTA0dDczM3Fsb3k4ZGI1N2wifQ.sDWh7-7xugNR7d109M6aFg'

In [402]:
df['text'] = 'Average price: ' +df['price'] + '<br>' +\
    'Overall Satisfaction '+df['overall_satisfaction']

In [415]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = "pk.eyJ1IjoiY21jNDU0IiwiYSI6ImNqZ2kyNzZoNTA0dDczM3Fsb3k4ZGI1N2wifQ.sDWh7-7xugNR7d109M6aFg"

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['38.9072'],
        lon=['-77.0369'],
        mode='markers',
        hoverinfo="text",
        text = df['text'],
    )
])
layout = graph_objs.Layout(
    height=600,
    margin=Margin(l=0, r=0, t=0, b=0),
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'dc-ward-map-overlay.geojson',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.7272,
            lon=-73.991251
        ),
        pitch=0,
        zoom=12,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ward-level-choropleths-python')